In [61]:
import re
import pandas as pd

Validate Regex : https://regex101.com/

# Data Cleaning

In [62]:
# Closing the file in important for preventing data access later and for other programs to access the file also.
f = open("_chat.txt")
data = f.read()
f.close()

In [63]:
data

'[15/12/19, 7:02:32 PM] Movie for every year 😴😴: \u200eMessages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them.\n[15/12/19, 7:02:32 PM] Nilay 2nd Floor: \u200eNilay 2nd Floor created this group\n[15/12/19, 7:02:32 PM] Movie for every year 😴😴: \u200eYou were added\n[10/04/22, 3:46:11 PM] Ashish Yadav Amity: Bhai bhai koi free hai its urgent 😦\n[10/04/22, 3:46:18 PM] Ashish Yadav Amity: Koi hai bhai 😵\n[10/04/22, 3:47:49 PM] Ritesh Amity: Mai hun bhai\n[10/04/22, 3:48:58 PM] Ashish Yadav Amity: Bhai mere system pe ransomware attack hogya hai\n[10/04/22, 3:49:05 PM] Ashish Yadav Amity: Kya kru ab 😭\n[10/04/22, 3:49:14 PM] Ritesh Amity: Ye kaise ho gaya\n[10/04/22, 3:49:26 PM] Ritesh Amity: Nilay ko call Karo\n[10/04/22, 3:49:30 PM] Ritesh Amity: Mujhe iska idea kam hai\n[10/04/22, 3:49:39 PM] Ashish Yadav Amity: Are yarr crack download kiya tha recovery software ka\n[10/04/22, 3:49:41 PM] Ritesh Amity: Bhai kahin SSD lagwaye 

In [64]:
# For AM, PM Formatted Time
pattern = "\[\d{1,2}/\d{1,2}/\d{1,2},\s\d{1,2}:\d{1,2}:\d{1,2}\s[APap][Mm]\]\s"

In [65]:
message = re.split(pattern, data)[1:]

In [66]:
message

['Movie for every year 😴😴: \u200eMessages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them.\n',
 'Nilay 2nd Floor: \u200eNilay 2nd Floor created this group\n',
 'Movie for every year 😴😴: \u200eYou were added\n',
 'Ashish Yadav Amity: Bhai bhai koi free hai its urgent 😦\n',
 'Ashish Yadav Amity: Koi hai bhai 😵\n',
 'Ritesh Amity: Mai hun bhai\n',
 'Ashish Yadav Amity: Bhai mere system pe ransomware attack hogya hai\n',
 'Ashish Yadav Amity: Kya kru ab 😭\n',
 'Ritesh Amity: Ye kaise ho gaya\n',
 'Ritesh Amity: Nilay ko call Karo\n',
 'Ritesh Amity: Mujhe iska idea kam hai\n',
 'Ashish Yadav Amity: Are yarr crack download kiya tha recovery software ka\n',
 'Ritesh Amity: Bhai kahin SSD lagwaye the un logon ne kuchh to nahi kiya\n',
 'Ritesh Amity: Are bhai 🤦🏻\u200d♂️\n',
 'Ashish Yadav Amity: Bhai pata nhi yarr\n',
 'Ritesh Amity: Kya aa raha hai?\n',
 'Ashish Yadav Amity: Bhai google account log out kr de rha bar bar\n',
 'Rit

In [67]:
dates = re.findall(pattern, data)

In [68]:
dates

['[15/12/19, 7:02:32 PM] ',
 '[15/12/19, 7:02:32 PM] ',
 '[15/12/19, 7:02:32 PM] ',
 '[10/04/22, 3:46:11 PM] ',
 '[10/04/22, 3:46:18 PM] ',
 '[10/04/22, 3:47:49 PM] ',
 '[10/04/22, 3:48:58 PM] ',
 '[10/04/22, 3:49:05 PM] ',
 '[10/04/22, 3:49:14 PM] ',
 '[10/04/22, 3:49:26 PM] ',
 '[10/04/22, 3:49:30 PM] ',
 '[10/04/22, 3:49:39 PM] ',
 '[10/04/22, 3:49:41 PM] ',
 '[10/04/22, 3:49:48 PM] ',
 '[10/04/22, 3:49:55 PM] ',
 '[10/04/22, 3:50:06 PM] ',
 '[10/04/22, 3:50:21 PM] ',
 '[10/04/22, 3:50:53 PM] ',
 '[10/04/22, 3:51:03 PM] ',
 '[10/04/22, 3:51:25 PM] ',
 '[10/04/22, 3:51:56 PM] ',
 '[10/04/22, 3:54:30 PM] ',
 '[10/04/22, 3:54:52 PM] ',
 '[10/04/22, 3:55:15 PM] ',
 '[10/04/22, 3:55:29 PM] ',
 '[10/04/22, 3:57:39 PM] ',
 '[10/04/22, 3:57:47 PM] ',
 '[10/04/22, 3:58:06 PM] ',
 '[10/04/22, 3:59:50 PM] ',
 '[10/04/22, 4:00:15 PM] ',
 '[10/04/22, 4:00:16 PM] ',
 '[10/04/22, 4:00:41 PM] ',
 '[10/04/22, 4:00:48 PM] ',
 '[10/04/22, 4:01:06 PM] ',
 '[10/04/22, 4:01:07 PM] ',
 '[10/04/22, 4:01:19

In [69]:
df = pd.DataFrame({'user_message':message, 'date':dates})

- %d matches the day of the month.
- %m matches the month.
- %y matches the two-digit year.
- %I matches the hour (12-hour clock).
- %M matches the minutes.
- %S matches the seconds.
- %p matches "AM" or "PM" in a case-insensitive manner.

In [70]:
df['date'] = pd.to_datetime(df['date'],format='[%d/%m/%y, %I:%M:%S %p] ')

In [71]:
df

,user_message,date
0,Movie for every year 😴😴: ‎Messages and calls a...,2019-12-15 19:02:32
1,Nilay 2nd Floor: ‎Nilay 2nd Floor created this...,2019-12-15 19:02:32
2,Movie for every year 😴😴: ‎You were added\n,2019-12-15 19:02:32
3,Ashish Yadav Amity: Bhai bhai koi free hai its...,2022-04-10 15:46:11
4,Ashish Yadav Amity: Koi hai bhai 😵\n,2022-04-10 15:46:18
...,...,...
17879,Aayush: Agle week tk rhoge kya\n,2023-10-02 10:32:43
17880,Ashish Yadav Amity: Haa next Sunday Tak hu\n,2023-10-02 10:58:38
17881,Ashish Yadav Amity: 😁✌🏻\n,2023-10-02 10:58:40
17882,Aayush: 👍👍\n,2023-10-02 11:44:57


In [72]:
df.shape

(17884, 2)

In [73]:
# Separting user and message
users = []
messages = []
for message in df['user_message']:
    line = re.split('([\w\W]+?):\s', message)
    # print(line[2])
    users.append(line[1])
    messages.append(line[2])
df['user'] = users
df['message'] = messages
df.drop(columns='user_message', inplace=True)

In [74]:
df.head()

,date,user,message
0,2019-12-15 19:02:32,Movie for every year 😴😴,‎Messages and calls are end-to-end encrypted. ...
1,2019-12-15 19:02:32,Nilay 2nd Floor,‎Nilay 2nd Floor created this group\n
2,2019-12-15 19:02:32,Movie for every year 😴😴,‎You were added\n
3,2022-04-10 15:46:11,Ashish Yadav Amity,Bhai bhai koi free hai its urgent 😦\n
4,2022-04-10 15:46:18,Ashish Yadav Amity,Koi hai bhai 😵\n


In [75]:
df.isnull().sum()

date       0
user       0
message    0
dtype: int64

In [81]:
#Extracting Years, Months, Days
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month_name()
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

In [82]:
df

,date,user,message,year,month,day,hour,minute
0,2019-12-15 19:02:32,Movie for every year 😴😴,‎Messages and calls are end-to-end encrypted. ...,2019,December,15,19,2
1,2019-12-15 19:02:32,Nilay 2nd Floor,‎Nilay 2nd Floor created this group\n,2019,December,15,19,2
2,2019-12-15 19:02:32,Movie for every year 😴😴,‎You were added\n,2019,December,15,19,2
3,2022-04-10 15:46:11,Ashish Yadav Amity,Bhai bhai koi free hai its urgent 😦\n,2022,April,10,15,46
4,2022-04-10 15:46:18,Ashish Yadav Amity,Koi hai bhai 😵\n,2022,April,10,15,46
...,...,...,...,...,...,...,...,...
17879,2023-10-02 10:32:43,Aayush,Agle week tk rhoge kya\n,2023,October,2,10,32
17880,2023-10-02 10:58:38,Ashish Yadav Amity,Haa next Sunday Tak hu\n,2023,October,2,10,58
17881,2023-10-02 10:58:40,Ashish Yadav Amity,😁✌🏻\n,2023,October,2,10,58
17882,2023-10-02 11:44:57,Aayush,👍👍\n,2023,October,2,11,44
